# 处理步骤
特征工程实际上就是对数据的处理，在将数据送入分类模型之前，需要将数据转换成合适的形式

- 数据预处理：对数据的基本处理操作
    - 填充缺失值
    - 处理时间格式的数据
    - 转换对象（非数值）类型特征到数值特征
- 异常值处理：？
    - 基于3segama原则
    - 基于箱型图
- 数据分箱：将数据按照预定的格式划分取值
    - 固定宽度分箱
    - 分位数分箱
        - 离散数值型数据分箱
        - 连续数值型数据分箱
    - 卡方分箱？
- 特征交互：？
    - 特征和特征之间组合
    - 特征和特征之间衍生
- 特征编码
    - one-hot编码
    - label-encode编码
- 特征选择：？
    - Filter
    - Wrapper（RFE）
    - Embedded

# 数据预处理

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder  
from sklearn.preprocessing import MinMaxScaler  # ?
from sklearn.feature_selection import SelectKBest  # ?
from sklearn.feature_selection import chi2  # ?
from sklearn.model_selection import StratifiedKFold  # ?
from sklearn.model_selection import KFold  # ?
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss

import xgboost as xgb  # ?
import lightgbm as lgb  # ?
from catboost import CatBoostRegressor  # ?

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('testA.csv')
df_train.shape, df_test.shape

((800000, 47), (200000, 48))

In [3]:
numerical_feats = df_train.select_dtypes(exclude=['object']).columns.tolist()
category_feats = df_train.select_dtypes(include=['object']).columns.tolist()
label = 'isDefault'
numerical_feats.remove(label)

## 填充缺失值
因为各个特征的缺失比例较低，选择填充的方式弥补缺失值，填充方法有：
- 固定填充
    ```
    df_train = df_train.fillna(0)  # 也可以对指定列进行固定填充，也可以采用均值填充
    ```
- 以上填充：用缺失值上面的值填充缺失值
    ```
    df_train = df_train.fillna(axis=0, method='ffill')
    ```
- 以下填充：用缺失值下面的值填充缺失值
    ```
    df_train = df_train.fillna(axis=0, method='bfill',limit=2)  # 限制最多只填充两个连续缺失值
    ```

In [4]:
df_train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           1
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  1
regionCode                0
dti                     239
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies      405
revolBal                  0
revolUtil               531
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     1
policyCode                0
n0                    40270
n1                    40270
n2                    40270
n2.1                

In [ ]:
# 对数值类型用中位数填充
df_train[numerical_feats] = 